## 03_01 Tokenisation

La tokenisation fait référence à la conversion d'une chaîne de texte en jetons individuels. Les jetons peuvent être des mots ou des ponctuations

In [16]:
import nltk
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import silhouette_score
import plotly.express as px
import plotly.graph_objects as go
from sklearn.cluster import KMeans


#Lire le fichier de base dans une variable de texte brut
dfs = pd.read_excel("/content/data.xlsx", sheet_name=None)
df2019 = dfs['2019']

# Convertir les dates en format datetime et normaliser
df2019['Date de remplissage de la fiche'] = pd.to_datetime(df2019['Date de remplissage de la fiche'], errors='coerce').dt.normalize()

df2019['Date de remplissage de la fiche'] = df2019['Date de remplissage de la fiche'].fillna(df2019['Date de remplissage de la fiche'].mode()[0])
df2019['Date de naissance'] = pd.to_datetime(df2019['Date de naissance'], errors='coerce').dt.normalize()
df2019['Date de naissance'] = df2019['Date de naissance'].fillna(df2019['Date de naissance'].mode()[0])
df2019['Si oui preciser la date du dernier don.'] = pd.to_datetime(df2019['Si oui preciser la date du dernier don.'], errors='coerce').dt.normalize()
# Calculer l'âge s

df2019['Age'] = df2019.apply( lambda x: x['Date de remplissage de la fiche'].year - x['Date de naissance'].year ,
    axis=1
).astype('Int64')

# Afficher les 5 premières valeurs de la colonne Age
ages = df2019.pop('Age')
df2019.insert(0,'Age',ages)
df2 = df2019.copy()
#print(df2019[['Age','Date de remplissage de la fiche','Date de naissance']].head(15))
df2019 = df2019.drop('Date de naissance', axis =1)
df_vol = dfs['Volontaire']
df_vol = df_vol.drop(['ID','Horodateur'],axis = 1)
df_vol['Date de remplissage de la fiche'] = np.nan
df_vol.columns = df2019.columns


df = pd.concat([df_vol,df2019],ignore_index = True)
imputer = SimpleImputer(strategy = 'constant',fill_value = 0)
df.columns = ['Date de remplissage de la fiche','Age',"Niveau d'etude", 'Genre', 'Taille', 'Poids',
       'Situation Matrimoniale (SM)', 'Profession',
       'Arrondissement de résidence', 'Quartier de Résidence', 'Nationalité',
       'Religion', 'A-t-il (elle) déjà donné le sang',
       'Si oui preciser la date du dernier don.', 'Taux d’hémoglobine',
       'ÉLIGIBILITÉ AU DON.',
       'Est sous anti-biothérapie',
       'Taux d’hémoglobine bas',
       'date de dernier Don < 3 mois',
       'IST récente (Exclu VIH, Hbs, Hcv)',
       'DDR',
       'DDR<14',
       'Allaitement',
       'A accoucher ces 6 derniers mois',
       'Interruption de grossesse  ces 06 derniers mois',
       'est enceinte',
       'Autre raisons ,preciser', 'Sélectionner "ok" pour envoyer',
       'Antécédent de transfusion',
       'Porteur(HIV,hbs,hcv)',
       'Opéré',
       'Drepanocytaire',       'Diabétique',
       'Hypertendus',
       'Asthmatiques',
       'Cardiaque',
       'Tatoué',
       'Scarifié',
       'Si autres raison preciser']


col = ['ÉLIGIBILITÉ AU DON.',
       'Est sous anti-biothérapie',
       'Taux d’hémoglobine bas',
       'date de dernier Don < 3 mois',
       'IST récente (Exclu VIH, Hbs, Hcv)',
       'DDR<14',
       'Allaitement',
       'A accoucher ces 6 derniers mois',
       'Interruption de grossesse  ces 06 derniers mois',
       'est enceinte',
       'Antécédent de transfusion',
       'Porteur(HIV,hbs,hcv)',
       'Opéré',
       'Drepanocytaire',
       'Diabétique',
       'Hypertendus',
       'Asthmatiques',
       'Cardiaque',
       'Tatoué',
       'Scarifié'
       ]
for c in col[1:] :
    df[c] = df[c].apply(lambda x : 1 if x == 'Oui' else 0 )
df['ÉLIGIBILITÉ AU DON.'] = df['ÉLIGIBILITÉ AU DON.'].apply(lambda x : 1 if x == 'Eligible' else 2 if x == 'Temporairement Non-eligible' else 0 )
df_cat = df[col]
col = df_cat.columns
df_cat = imputer.fit_transform(df_cat)
df_cat = pd.DataFrame(df_cat, columns = col)

<ipython-input-16-509f6f761b55>:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df2019['Date de remplissage de la fiche'] = pd.to_datetime(df2019['Date de remplissage de la fiche'], errors='coerce').dt.normalize()


In [34]:


df2 = df.copy()
va = list(df2.columns)
df_autres = df2[['Date de remplissage de la fiche','Autre raisons ,preciser',  'Si autres raison préciser']]
#Extraction des tokens
#Autres raisons
df_autres['tokens indisponible']  =df_autres['Autre raisons ,preciser'].apply(nltk.word_tokenize)
df_autres['tokens ineligible']  =df_autres['Si autres raison préciser'].apply(nltk.word_tokenize)
print("Token List : ",df_autres.head())
print("\n Total Tokens : ",df_autres.shape[0])

TypeError: expected string or bytes-like object, got 'float'

In [31]:
df2019.columns

Index(['Age', 'Date de remplissage de la fiche', 'Niveau d'etude', 'Genre',
       'Taille', 'Poids', 'Situation Matrimoniale (SM)', 'Profession',
       'Arrondissement de résidence', 'Quartier de Résidence', 'Nationalité',
       'Religion', 'A-t-il (elle) déjà donné le sang',
       'Si oui preciser la date du dernier don.', 'Taux d’hémoglobine',
       'ÉLIGIBILITÉ AU DON.',
       'Raison indisponibilité  [Est sous anti-biothérapie  ]',
       'Raison indisponibilité  [Taux d’hémoglobine bas ]',
       'Raison indisponibilité  [date de dernier Don < 3 mois ]',
       'Raison indisponibilité  [IST récente (Exclu VIH, Hbs, Hcv)]',
       'Date de dernières règles (DDR) ',
       'Raison de l’indisponibilité de la femme [La DDR est mauvais si <14 jour avant le don]',
       'Raison de l’indisponibilité de la femme [Allaitement ]',
       'Raison de l’indisponibilité de la femme [A accoucher ces 6 derniers mois  ]',
       'Raison de l’indisponibilité de la femme [Interruption de gros

In [36]:
"".isinstance('float')

AttributeError: 'str' object has no attribute 'isinstance'

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt_tab')  # Téléchargement du tokenizer de NLTK

# Exemple de DataFrame
df = pd.DataFrame({'text': ["Bonjour, comment ça va ?", "Je fais du NLP avec Python !"]})

# Tokenisation
df['tokens'] = df['text'].apply(word_tokenize)

print(df)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


                           text                                tokens
0      Bonjour, comment ça va ?      [Bonjour, ,, comment, ça, va, ?]
1  Je fais du NLP avec Python !  [Je, fais, du, NLP, avec, Python, !]


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## 03_02 Nettoyage du texte

Nous verrons des exemples de suppression de ponctuation et de conversion en minuscules

#### Supprimer la ponctuation

In [ ]:
#Utiliser la bibliothèque Punkt pour extraire les jetons
token_list2 = list(filter(lambda token: nltk.tokenize.punkt.PunktToken(token).is_non_punct, token_list))
print("Liste des tokens après suppression de la ponctuation : ",token_list2[:20])
print("\nTotal des token après suppression de la ponctuation : ", len(token_list2))

NameError: name 'token_list' is not defined

#### Convertir en minuscules

In [ ]:
token_list3=[word.lower() for word in token_list2 ]
print("Liste des tokens après conversion en minuscules : ", token_list3[:20])
print("\nTotal des tokens après conversion en minuscules : ", len(token_list3))

Liste des tokens après conversion en minuscules :  ['si', 'vous', 'vous', 'intéressez', 'au', 'big', 'data', 'vous', 'connaissez', 'certainement', 'apache', 'spark', 'savez-vous', 'pourquoi', 'spark', 'est', 'le', 'framework', 'de', 'prédilection']

Total des tokens après conversion en minuscules :  87


## 03_03 Suppression des mots vides

Suppression des mots vides à l'aide d'une liste de mots vides standard disponible dans NLTK pour l'anglais

In [ ]:
#Download the standard stopword list
nltk.download('stopwords')
from nltk.corpus import stopwords

#Remove stopwords
token_list4 = list(filter(lambda token: token not in stopwords.words('french'), token_list3))
print("Liste de tokens après suppression des mots vides : ", token_list4[:20])
print("\nTotal de tokens après suppression des mots vides : ", len(token_list4))

Liste de tokens après suppression des mots vides :  ['si', 'intéressez', 'big', 'data', 'connaissez', 'certainement', 'apache', 'spark', 'savez-vous', 'pourquoi', 'spark', 'framework', 'prédilection', 'traitement', 'données', 'massives', 'pourquoi', 'est-il', 'autant', 'apprécié']

Total de tokens après suppression des mots vides :  54


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## 03_04 Stemming

In [ ]:
#Utilisez la bibliothèque SnowballStemmer pour la radicalisation.
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("french")

#Stem data
token_list5 = [stemmer.stem(word) for word in token_list4 ]
print("Liste de tokens après le stemming : ", token_list5[:20])
print("\nTotal de tokens après Stemming : ", len(token_list5))

Liste de tokens après le stemming :  ['si', 'intéress', 'big', 'dat', 'connaiss', 'certain', 'apach', 'spark', 'savez-vous', 'pourquoi', 'spark', 'framework', 'prédilect', 'trait', 'don', 'massiv', 'pourquoi', 'est-il', 'aut', 'appréci']

Total de tokens après Stemming :  54


## 03_05 Lemmatisation

In [ ]:
#Utilisez la bibliothèque wordnet pour mapper les mots à leur forme lemmatisée
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
token_list6 = [lemmatizer.lemmatize(word) for word in token_list4 ]
print("Liste de tokens après lemmatisation : ", token_list6[:20])
print("\nTotal des tokens après lemmatisation : ", len(token_list6))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Liste de tokens après lemmatisation :  ['si', 'intéressez', 'big', 'data', 'connaissez', 'certainement', 'apache', 'spark', 'savez-vous', 'pourquoi', 'spark', 'framework', 'prédilection', 'traitement', 'données', 'massif', 'pourquoi', 'est-il', 'autant', 'apprécié']

Total des tokens après lemmatisation :  54


#### Comparaison des tokens entre stemming et lemmatisation

In [ ]:
#Vérifier les technologies de jeton
print( "Raw : ", token_list4[20]," , Stemmed : ", token_list5[20], " , Lemmatized : ", token_list6[20])


Raw :  notamment  , Stemmed :  not  , Lemmatized :  notamment


### Lemmatisation du text en français

In [ ]:
# Plus d'information sur ce package : https://github.com/sammous/spacy-lefff


# Pour installer spacy usiliser la commande "pip install spacy"
# Il faut installer le dictionnaire français sur spacy "python -m spacy download fr"
# Pour installer spacy lemmatiser utiliser la commande "pip install spacy-lefff"
import spacy
from spacy.language import Language


nlp = spacy.load('fr_core_news_sm')
# nlp.add_pipe('french_lemmatizer', name='lefff')
doc = nlp(" ".join(token_list4))
for d in doc:
    print(d.text, d.lemma_)

si si
intéressez intéressez
big big
data data
connaissez connaître
certainement certainement
apache apache
spark spark
savez savoir
- -
vous vous
pourquoi pourquoi
spark spark
framework framework
prédilection prédilection
traitement traitement
données donnée
massives massif
pourquoi pourquoi
est-il est-il
autant autant
apprécié apprécier
notamment notamment
déployer déployer
algorithmes algorithme
machine machine
learning learning
découvrez découvrir
cours cours
apache apache
pyspark pyspark
répondre répondre
toutes tout
questions question
travers travers
multiples multiple
exemples exemple
mises mise
pratique pratique
professeur professeur
associé associer
technologies technologie
l' le
information information
techniques technique
d' de
optimisation optimisation
donne donne
toutes tout
clés clé
analyser analyser
efficacement efficacement
données donner
grande grand
échelle échelle
apache apache
spark spark
python python
